In [1]:
# В РЕАЛЬНОМ ВРЕМЕНИ ОПОЗНАЕТ ЖЕСТЫ ОБЕИХ РУК (MLP) МОДЕЛЬ И MAXMASS БЕРЕТ ИЗ ФАЙЛОВ

import cv2
import mediapipe as mp
import time
import math
from google.protobuf.json_format import MessageToDict
import numpy as np
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import json

In [2]:
class Dataset(Dataset):
    def __init__(self, *args): #получаем n массивов данных
        cnt, labels, temp =0, [], []
        self.data = []
        for data in args: #выделяем по одному из них
            self.data.extend(torch.tensor(data).type(torch.float32)) # преобразуем каждый в тензор и добавляем к общим данным
            print(len(data))
            for j in range(len(data)):
                labels.append(cnt) #присовим номера каждому подмассиву для проверки
            cnt += 1
        self.labels = labels
            
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.data)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(42, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 2)
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x),negative_slope=0.001)
        x = F.leaky_relu(self.fc2(x),negative_slope=0.001)
        x = self.fc4(x)
        return x

In [3]:
path_to_lmodel = "D:/BMM/models/l_model.pth"
path_to_lmaxmass = "D:/BMM/maxmasses/l_maxmass.npy"
path_to_rmodel = "D:/BMM/models/r_model.pth" 
path_to_rmaxmass = "D:/BMM/maxmasses/r_maxmass.npy"

r_net = torch.load(path_to_rmodel) 
r_net.eval()
l_net = torch.load(path_to_lmodel) 
l_net.eval()

r_maxmass = np.load(path_to_rmaxmass, allow_pickle=True)
l_maxmass = np.load(path_to_lmaxmass, allow_pickle=True)

In [4]:
class handDetector():
    def __init__(self, mode=False, maxHands=3, modelComplexity=1, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.modelComplexity = modelComplexity
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.modelComplexity, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return (img , self.results.multi_hand_landmarks, self.results.multi_handedness)

    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                xList.append(cx)
                yList.append(cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255,0,255), cv2.FILLED)
            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax
            bbox = list(bbox)
            a = xmax - xmin
            b = ymax - ymin
            if len(bbox) == 4:
                if a >b:
                    if bbox[0] -((a - b) //2 ) > 0:
                        razn = (a - b) //2 
                        bbox[3] = bbox[3] + razn
                        bbox[0] = bbox[0] - razn           
                else:
                    if bbox[1] -((b - a) //2  ) > 0:
                        razn = (b - a) //2 
                        bbox[2] = bbox[2] + razn
                        bbox[1] = bbox[1] - razn
            if draw:
                cv2.rectangle(img, (bbox[0]-20, bbox[1]-20), (bbox[2]+20, bbox[3]+20), (0, 255, 0), 2)
        return self.lmList, bbox

    def findDistance(self, p1, p2, img, draw=True):
        x1, y1 = self.lmList[p1][1], self.lmList[p1][2]
        x2, y2 = self.lmList[p2][1], self.lmList[p2][2]
        cx, cy = (x1+x2)//2, (y1+y2)//2

        if draw:
            cv2.circle(img, (x1,y1), 15, (255,0,255), cv2.FILLED)
            cv2.circle(img, (x2,y2), 15, (255,0,255), cv2.FILLED)
            cv2.line(img, (x1,y1), (x2,y2), (255,0,255), 3)
            cv2.circle(img, (cx,cy), 15, (255,0,255), cv2.FILLED)

        length = math.hypot(x2-x1, y2-y1)
        return length, img, [x1, y1, x2, y2, cx, cy]

    def fingersUp(self):
        fingers = []

        # Thumb
        if self.lmList[self.tipIds[0]][1] < self.lmList[self.tipIds[0]-1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # 4 Fingers
        for id in range(1,5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        return fingers

def cords_to_vers(lands, hand): #МАССИВ ИЗ 42 КООРДИНАТ В МАСИВ ПРЕДСКАЗАНИЙ ЖЕСТОВ
    if hand == 'r':
        net = r_net
    elif hand == 'l':
        net = l_net
    pred2 = []
    b = []
    for id, lm in enumerate(lands.landmark):
        h,w,c = img.shape
        cx, cy = int(lm.x*w), int(lm.y*h)
        b.append(lm.x)
        b.append(lm.y)
        if id == 8:
            cx8 = cx
        if  id == 8 or id == 12:
            cv2.circle(img, (cx,cy),10,(255,0,255),cv2.FILLED)
    flag = 0
    if len(b) == 42:
        pred = net(torch.tensor(b))
        pred2 = pred.detach().numpy()
    else:
        flag = 1
    detector.mpDraw.draw_landmarks(img, lands, detector.mpHands.HAND_CONNECTIONS)
    return(pred2, flag)

def vers_to_gest(pred, flag, hand): #МАССИВ ПРЕДСКАЗАНИЙ В ЖЕСТ
    gests = os.listdir("D:/BMM/datasets/"   + hand + "_dataset")
    if hand == 'r':
        maxmass = r_maxmass
    elif hand == 'l':
        maxmass = l_maxmass
    ans = 'nogest'
    exist = [float(i) for i in pred]
    if len(exist) == 2:
        tempans = max(exist)
        tempansid = exist.index(tempans)
        if flag ==0:    
            if tempans >= maxmass[tempansid]: # - (maxmass[tempansid])/10: #МЕТРИКИ?
                ans = gests[tempansid][:-4]
            else:
                ans = 'nogest'
    return(ans) 
    

    


In [5]:
pTime = 0
cTime = 0
cap = cv2.VideoCapture(0)
detector = handDetector()
j = 0

while True:
    #if j ==5:
    #    break
    #j+=1
    success, img = cap.read()
    img = cv2.flip(img,1) # Mirror flip  
    img, lands, handed = detector.findHands(img)
    lmList = detector.findPosition(img)
    if lands:
        if len(handed) == 2:
            for i in handed:
                label = MessageToDict(i)['classification'][0]['label']
                if label == 'Left':
                    sdvig = 20
                    hand = 'l'
                    if handflag == 0:
                        jhand = lands[0]
                    else: 
                        jhand = lands[1]
                elif label == 'Right':
                    sdvig = 460
                    hand = 'r'
                    if handflag == 0:
                        jhand = lands[1]
                    else: 
                        jhand = lands[0]                   
                cords, flag = cords_to_vers(jhand,hand)
                gest = vers_to_gest(cords, flag, hand)
                cv2.putText(img, gest, (sdvig, 50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (0, 255, 0), 2)       
        else:
            for i in handed:
                label = MessageToDict(i)['classification'][0]['label']
                if label == 'Left':
                    handflag = 1
                    cords, flag = cords_to_vers(lands[0],'l')
                    gest = vers_to_gest(cords, flag, 'l')
                    cv2.putText(img, gest, (20, 50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (0, 255, 0), 2)
                if label == 'Right':
                    handflag = 0
                    cords, flag = cords_to_vers(lands[0],'r')
                    gest = vers_to_gest(cords, flag, 'r')
                    cv2.putText(img, gest, (460, 50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (0, 255, 0), 2)      
    cTime = time.time()
    fps = 1. / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, str(int(fps)), (10,90), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255), 3)
    cv2.imshow("Image", img)
    cv2.waitKey(1)
    if cv2.waitKey(20) == 27: # exit on ESC
        break
cv2.destroyWindow("Image")
cap.release()
cv2.waitKey(1)

-1